Anderson Pepple CA1 Submission

In [56]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog

login_details = {
    "admin": "123",
}

def authenticate_user(user, password):
    return login_details.get(user) == password

net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
classes = []
with open('cfg/coco.names', 'r') as f:
    classes = f.read().splitlines()

def select_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4")])
    if file_path:
        process_video(file_path)

def process_video(file_path):
    cap = cv2.VideoCapture(file_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        height, width, _ = frame.shape

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)

        output_layers_names = net.getUnconnectedOutLayersNames()
        layers_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layers_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord(' '):
            break

    cap.release()
    cv2.destroyAllWindows()

def open_new_window():
    def select_and_display_video():
        file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4")])
        if file_path:
            file_path_label.config(text="Selected Video: " + file_path)

    def perform_object_detection():
        video_path = file_path_label.cget("text")
        if video_path.startswith("Selected Video: "):
            video_path = video_path.replace("Selected Video: ", "")
            process_video(video_path)

    root1 = tk.Tk()
    root.destroy()

    root1.title("Object Detection Options")
    root1.geometry("610x270")

    space = tk.Label(root1, text="\n", font=("Helvetica", 10))
    space.pack()

    file_path_label = tk.Label(root1, text="No video selected", font=("Helvetica", 10))
    file_path_label.pack()

    space = tk.Label(root1, text=" \n", font=("Helvetica", 10))
    space.pack()

    select_button = tk.Button(root1, text="Select Video", command=select_and_display_video)
    select_button.pack(pady=10)

    detect_button = tk.Button(root1, text="Perform Object Detection", command=perform_object_detection)
    detect_button.pack(pady=5)

    close_label = tk.Label(root1, text="Click spacebar to close video", font=("Helvetica", 10), fg="red")
    close_label.pack()

    root1.mainloop()

   

def login():
    user = user_entry.get()
    password = password_entry.get()

    if authenticate_user(user, password):
        
        open_new_window()
    else:
        login_status_label.config(text="Invalid email or password", fg="red")

# Creating window
root = tk.Tk()
root.title("Anderson's Video Object Detection App")
root.geometry("410x270")

logo_img = tk.PhotoImage(file="PAUlogo1.png")
logo_label = tk.Label(root, image=logo_img)
logo_label.pack()

inst = tk.Label(root, text="Login to select a video for detection", font=("Helvetica", 10))
inst.pack()

space = tk.Label(root, text=" ", font=("Helvetica", 10))
space.pack()

# username Entry
user_label = tk.Label(root, text="Username:")
user_label.pack()
user_entry = tk.Entry(root, font=("Helvetica", 12))
user_entry.pack()

# Password Entry
password_label = tk.Label(root, text="Password:")
password_label.pack()
password_entry = tk.Entry(root, show="*", font=("Helvetica", 12))
password_entry.pack()

space = tk.Label(root, text=" ", font=("Helvetica", 10))
space.pack()

# Login Button
login_button = tk.Button(root, text="Login", command=login, font=("Helvetica", 12), bg="#4CAF50", fg="white", relief=tk.FLAT)
login_button.pack(pady=10)

# Login Status Label
login_status_label = tk.Label(root, text="")
login_status_label.pack()

close = tk.Label(root, text="click spacebar to close video", font=("Helvetica", 8))
close.pack()

root.mainloop()
